## 네이버뉴스 언론사/일자별 동적 웹크롤링

In [4]:
import time
import pandas as pd  # 데이터프레임 및 CSV 저장을 위해 추가
from datetime import datetime # 파일명에 시간을 넣기 위해 추가
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====================================================
# 1. 설정 및 드라이버 초기화
# ====================================================
def set_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 브라우저 안 띄우고 하려면 주석 해제
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# ====================================================
# 2. 기사 상세 내용 추출 함수 (업그레이드 버전)
# ====================================================
def extract_article_info(driver, url):
    try:
        driver.get(url)
        # 제목이 뜰 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span"))
        )
        time.sleep(0.5) 

        # 1) 제목
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except:
            title = "제목 없음"

        # 2) 본문 (캡션 제거 포함)
        try:
            # 본문 영역 찾기
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")
            
            # 자바스크립트로 불필요한 요소(이미지 캡션, 요약봇 등) 강제 삭제
            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary"); 
                for (var i = 0; i < dirts.length; i++) {
                    dirts[i].remove();
                }
            """, dic_area)
            
            content = dic_area.text
            content = content.replace("\n", " ").strip() # 줄바꿈 정리
        except:
            content = "본문 없음"

        # 3) 게재 시간
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, 
                "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span"
            )
            date_text = date_element.text
        except:
            try:
                date_text = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except:
                date_text = "날짜 없음"

        print(f"[수집] {title} ({date_text})")
        # 제목, 본문, 날짜, 링크를 모두 담아서 반환
        return {"제목": title, "본문": content, "날짜": date_text, "링크": url}

    except Exception as e:
        print(f"[에러] 기사 상세 수집 중 오류: {url} -> {e}")
        return None

# ====================================================
# 3. 메인 크롤링 로직
# ====================================================
def main():
    driver = set_chrome_driver()
    
    # [설정] 시작 URL
    target_url = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=008&date=20251219"
    
    driver.get(target_url)
    time.sleep(2)

    collected_data = [] # 데이터를 모을 리스트
    page_count = 1

    try:
        while True:
            print(f"\n===== 현재 리스트 페이지: {page_count} =====")
            
            # Step A: URL 수집
            article_urls = []
            selectors = [
                "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
                "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
            ]

            for sel in selectors:
                links = driver.find_elements(By.CSS_SELECTOR, sel)
                for link in links:
                    url = link.get_attribute("href")
                    if url:
                        article_urls.append(url)
            
            print(f"-> 수집할 기사 수: {len(article_urls)}개")

            # Step B: 상세 페이지 순회
            list_window = driver.current_window_handle
            
            for url in article_urls:
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[-1])
                
                data = extract_article_info(driver, url)
                if data:
                    collected_data.append(data)
                
                driver.close()
                driver.switch_to.window(list_window)
            
            # Step C: 페이징 이동
            try:
                paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")
                current_page_elem = paging_area.find_element(By.TAG_NAME, "strong")
                current_page_num = int(current_page_elem.text)
                next_page_num = current_page_num + 1
                
                next_page_btn = None
                try:
                    next_page_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_page_num}']")
                except:
                    next_page_btn = None
                
                if next_page_btn:
                    print(f"-> {next_page_num} 페이지로 이동합니다.")
                    next_page_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue

                try:
                    next_group_btn = paging_area.find_element(By.CSS_SELECTOR, "a.next")
                    print("-> '다음' 그룹(Next) 버튼을 클릭합니다.")
                    next_group_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue
                except:
                    print("-> 더 이상 다음 페이지가 없습니다. 수집 종료.")
                    break

            except Exception as e:
                print(f"-> 페이징 처리 중 에러 혹은 종료: {e}")
                break
                
    except KeyboardInterrupt:
        print("\n[사용자 중단] 크롤링을 강제로 중단합니다.")
    
    finally:
        driver.quit()
        
        # ====================================================
        # [수정됨] 4. 데이터 저장 (파일명 URL 기반 생성)
        # ====================================================
        print("\n[저장 시작] 데이터를 CSV 파일로 저장합니다...")
        
        if collected_data:
            df = pd.DataFrame(collected_data)
            
            # --- [파일명 수정 부분] ---
            try:
                # URL에서 'oid=' 뒷부분(예: 009&date=20251219)을 잘라내서 파일명으로 사용
                # target_url 예: ...&oid=009&date=20251219
                if "oid=" in target_url:
                    suffix = target_url.split("oid=")[1]  # '009&date=20251219' 추출
                    filename = f"oid={suffix}.csv"
                else:
                    # oid가 없는 URL일 경우 기존 방식(시간) 사용
                    raise ValueError("URL에 oid 파라미터가 없습니다.")
            except Exception:
                # 실패 시 안전하게 시간 기반 파일명 사용
                now = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"naver_news_result_{now}.csv"
            # ------------------------
            
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            
            print(f"✅ 저장 완료! 파일명: {filename}")
            print(f"총 {len(df)}개의 기사가 저장되었습니다.")
        else:
            print("❌ 수집된 데이터가 없어 파일을 저장하지 않았습니다.")

if __name__ == "__main__":
    main()


===== 현재 리스트 페이지: 1 =====
-> 수집할 기사 수: 20개
[수집] 남산 케이블카 '64년 독점' 안 깨졌다…공공 곤돌라 사업 '불투명' (2025.12.19. 오후 3:37)
[수집] 기부는 정체, 물가는 급등…'먹거리 안전망' 푸드뱅크 이중고 (2025.12.19. 오후 3:36)
[수집] [스팟]코스닥 13.94포인트(1.55%) 오른 915.27 마감 (2025.12.19. 오후 3:33)
[수집] 서울시 '남산 곤돌라 사업' 제동…법원 "용도구역 변경 취소" (2025.12.19. 오후 3:33)
[수집] 중국, 미 국채 보유규모 '17년 최소'…"달러 함정에 빠지지 않게" (2025.12.19. 오후 3:33)
[수집] [스팟]코스피 26.04포인트(0.65%) 오른 4020.55 마감 (2025.12.19. 오후 3:33)
[수집] 이재명 대통령, 공정위에 "조사 불응 기업에 경제제재 장치 마련하라" (2025.12.19. 오후 3:31)
[수집] 원/달러 환율 종가, 2원 내린 1476.3원 (2025.12.19. 오후 3:30)
[수집] 대기업 총수일가 부당내부거래 과징금 상향…정액과징금 40억→100억 (2025.12.19. 오후 3:30)
[수집] 공정위, 식품·교육·건설·에너지 등 4대분야 가격담합 집중점검 (2025.12.19. 오후 3:30)
[수집] 산재 작업중지 추가비용 발생시 하도급업체에 대금조정 신청권 부여 (2025.12.19. 오후 3:30)
[수집] "지선 전 대전·충남 통합" 與 속도전…野 "선거개입 안 돼" (2025.12.19. 오후 3:29)
[수집] 방미통위, 한국수어통역방송 품질개선 추진…미디어 접근권 제고 (2025.12.19. 오후 3:28)
[수집] 콜드플레이 콘서트서 '불륜' 들킨 여성 근황..."퇴사 후 살해 협박까지" (2025.12.19. 오후 3:28)
[수집] '롯데 신동주 불법자문' 민유성 전 산업은행장, 2심서 집유 (2025.12.19. 오후 3:26)
[수집] 성남시 분

[수집] KH건설, '싱크홀 막는' 부력방지공법 국토부 건설신기술 지정 (2025.12.19. 오후 12:44)
[수집] 검찰, 1억대 뒷돈받고 지점 입점시킨 기업은행 전 부행장 구속기소 (2025.12.19. 오후 12:29)
[수집] 홍은희 개인전 '사라'…19~25일 컷더케이크 갤러리 (2025.12.19. 오후 12:25)
[수집] [속보]일본 기준금리 0.5%→0.75% 인상…30년 만의 최고 (2025.12.19. 오후 12:22)
[수집] 김민석 총리 "북향민 자살, 내국민의 2배"…정동영 장관 "핵심은 일자리" (2025.12.19. 오후 12:07)
-> 7 페이지로 이동합니다.

===== 현재 리스트 페이지: 7 =====
-> 수집할 기사 수: 20개
[수집] 이재명 대통령 "북한 매체 접근 금지…국민을 선전·선동에 넘어간다고 보는 것" (2025.12.19. 오후 12:07)
[수집] 지재처·KAIST·특허법원, 국가 지식재산 경쟁력 강화 협력 MOU (2025.12.19. 오후 12:06)
[수집] 김포 파크골프장 논란…협회 "공공체육시설 전유물 될 수 없어" (2025.12.19. 오후 12:03)
[수집] 공정위, 3년 연속 '공공데이터 품질인증 최우수기관' 선정 (2025.12.19. 오후 12:00)
[수집] 나도 모르는 내 휴대폰이 개통됐다고?...'안면인증'으로 원천 차단 (2025.12.19. 오후 12:00)
[수집] H2O, UAE 스테이블코인 파트너 선정…"관광객 코인 활용 지원" (2025.12.19. 오후 12:00)
[수집] 의회 문턱 넘은 양주문화관광재단 설립…지역 문화·관광 활성화 '시동' (2025.12.19. 오전 11:57)
[수집] 올바른에프앤비·서울오라토리오, 삼척 도계지역 아동 위한 연말 나눔 음악회 개최 (2025.12.19. 오전 11:57)
[수집] 與 '대전·충남통합 특위' 구성…"선거 유불리 상관없다" (2025.12.19. 오전 11:50)
[수집] 이재명 대통령 "비전향 장기수, 본인 

[수집] "2028년까지 성장 전망"에 LIG넥스원 5%대 강세 (2025.12.19. 오전 9:22)
[수집] 서부T&D, 호텔 특수에 토지개발 기대감…목표가↑-NH (2025.12.19. 오전 9:21)
[수집] HL만도, 2035년까지 온실가스 최대 66% 감축 목표…SBTi 공식 승인 (2025.12.19. 오전 9:21)
[수집] 새해 일출 명소 1위는 속초 청초호…2위는? (2025.12.19. 오전 9:21)
[수집] HD현대, 안전 비전 공표…정기선 회장 "안전, 기업의 생존 조건" (2025.12.19. 오전 9:21)
[수집] "신규 수주 순항 중" 증권가 분석에 두산에너빌리티, 상승 (2025.12.19. 오전 9:21)
[수집] HLB에너지, 친환경 자원순환시설 '그린에너지파크' 준공 (2025.12.19. 오전 9:19)
[수집] 보수 야권 '통일교 특검' 논의 지속…송언석·천하람 일요일 오찬 회동 (2025.12.19. 오전 9:19)
-> 13 페이지로 이동합니다.

===== 현재 리스트 페이지: 13 =====
-> 수집할 기사 수: 20개
[수집] 식당 앞에서 바지 내린 외국인 여성…그대로 앉아 노상 방뇨 '충격' (2025.12.19. 오전 9:18)
[수집] IPO시장 훈풍에…알지노믹스 상한가 (2025.12.19. 오전 9:16)
[수집] '관봉권·쿠팡 특검', 한국은행 발권국 영장집행…첫 강제수사 (2025.12.19. 오전 9:16)
[수집] 코스피, 미국 증시 상승 힘입어 4000피 탈환…개인·기관 '사자' (2025.12.19. 오전 9:16)
[에러] 기사 상세 수집 중 오류: https://n.news.naver.com/mnews/article/008/0005294151 -> Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3312d3
	0x331314
	0x11e6dd
	0x1693a5
	0x16977b
	0x1b0382

[에러] 기사 상세 수집 중 오류: https://n.news.naver.com/mnews/article/008/0005294056 -> Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3312d3
	0x331314
	0x11e6dd
	0x1693a5
	0x16977b
	0x1b0382
	0x18b534
	0x1adb13
	0x18b2e6
	0x15d321
	0x15e1d4
	0x585314
	0x5808cb
	0x59d1aa
	0x34b1d8
	0x3531dd
	0x3395d8
	0x339799
	0x323b28
	0x75b9fcc9
	0x773982ae
	0x7739827e
	0

-> 18 페이지로 이동합니다.

===== 현재 리스트 페이지: 18 =====
-> 수집할 기사 수: 6개
[에러] 기사 상세 수집 중 오류: https://n.news.naver.com/mnews/article/008/0005294055 -> Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3312d3
	0x331314
	0x11e6dd
	0x1693a5
	0x16977b
	0x1b0382
	0x18b534
	0x1adb13
	0x18b2e6
	0x15d321
	0x15e1d4
	0x585314
	0x5808cb
	0x59d1aa
	0x34b1d8
	0x3531dd
	0x3395d8
	0x339799
	0x323b28
	0x75b9fcc9
	0x773982ae
	0x7739827e
	0

[에러] 기사 상세 수집 중 오류: https://n.news.naver.com/mnews/article/008/0005294054 -> Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x3312d3
	0x331314
	0